<a href="https://colab.research.google.com/github/elvissoares/COQ878-QuiQuaComp/blob/main/notebooks/notebook_Aula_MACE_Otimizando_Cristal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Otimização de Estrutura Cristalina usando MACE e ASE

Autor: [Prof. Elvis do A. Soares](https://github.com/elvissoares) 

Contato: [elvis@peq.coppe.ufrj.br](mailto:elvis@peq.coppe.ufrj.br) - [Programa de Engenharia Química, PEQ/COPPE, UFRJ, Brasil](https://www.peq.coppe.ufrj.br/)

---

## Instalando pacotes necessários

In [ ]:
!pip install --upgrade ase numpy==2.0 -q
!pip install git+https://github.com/imagdau/aseMolec@f411b5618381ba3b807cbbb041f12f460a69d606 -q
!pip install mace-torch cuequivariance cuequivariance-torch cuequivariance-ops-torch-cu12 torch-dftd -q

### Importando MACE_MP foundation model

In [1]:
from mace.calculators import mace_mp

macemp = mace_mp(model="small", dispersion=True, default_dtype = "float32", device="cuda")

/home/elvis/Programs/miniforge3/envs/mlff/lib/python3.10/site-packages/e3nn/o3/_wigner.py:10: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  _Jd, _W3j_flat, _W3j_indices = torch.load(os.path.join(os.path.dirname(__file__), 'constants.pt'))


Using Materials Project MACE for MACECalculator with /home/elvis/.cache/mace/20231210mace128L0_energy_epoch249model
Using float32 for MACECalculator, which is faster but less accurate. Recommended for MD. Use float64 for geometry optimization.


/home/elvis/Programs/miniforge3/envs/mlff/lib/python3.10/site-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


Using head Default out of ['Default']
Default dtype float32 does not match model dtype float64, converting models to float32.
Using TorchDFTD3Calculator for D3 dispersion corrections


### Criando a geometria da rede cristalina

Pode ser um arquivo cif, por exemplo

> atoms = read('blabla.cif')

In [2]:
!wget https://raw.githubusercontent.com/elvissoares/COQ878-QuiQuaComp/refs/heads/main/notebooks/mace/ice_Ih.cif -O ice_Ih.cif



7[Files: 0  Bytes: 0  [0 B/s] Re]87[https://raw.githubusercontent.]87Saving 'ice_Ih.cif'
87ice_Ih.cif           100% [=============================>]     659     --.-KB/s87HTTP response 200  [https://raw.githubusercontent.com/elvissoares/COQ878-QuiQuaComp/refs/heads/main/notebooks/mace/ice_Ih.cif]
87ice_Ih.cif           100% [=============================>]     659     --.-KB/s87[Files: 1  Bytes: 659  [1.38KB/]8

In [3]:
from ase.io import write, read

atoms = read('ice_Ih.cif')

### Definindo as simetrias a serem mantidas

In [4]:
from ase.constraints import FixSymmetry
from ase.filters import UnitCellFilter

atoms.set_constraint(FixSymmetry(atoms)) #Constraint to preserve spacegroup symmetry during optimisation.
atoms_filter = UnitCellFilter(atoms) #Modify the supercell and the atom positions.

### Definindo o método de otimização e a calculadora

In [5]:
atoms.calc = macemp

from ase.optimize import BFGS

opt = BFGS(atoms_filter)

### Calculando a otimização com BFGS

In [6]:
print('Initial Energy', atoms.get_potential_energy())
opt.run(fmax=0.001)
print('Final Energy', atoms.get_potential_energy())

Initial Energy -179.87988409872153
      Step     Time          Energy          fmax
BFGS:    0 12:02:42     -179.879884        0.173521


/home/elvis/Programs/miniforge3/envs/mlff/lib/python3.10/site-packages/torch_dftd/torch_dftd3_calculator.py:98: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  cell: Optional[Tensor] = torch.tensor(


BFGS:    1 12:02:42     -179.883149        0.081899
BFGS:    2 12:02:43     -179.885856        0.087021
BFGS:    3 12:02:43     -179.889602        0.072664
BFGS:    4 12:02:43     -179.890588        0.052369
BFGS:    5 12:02:43     -179.891402        0.046323
BFGS:    6 12:02:43     -179.892505        0.082545
BFGS:    7 12:02:43     -179.894109        0.100016
BFGS:    8 12:02:43     -179.895623        0.070164
BFGS:    9 12:02:44     -179.896853        0.067680
BFGS:   10 12:02:44     -179.898138        0.063796
BFGS:   11 12:02:44     -179.899843        0.097086
BFGS:   12 12:02:44     -179.901481        0.098843
BFGS:   13 12:02:44     -179.902346        0.052016
BFGS:   14 12:02:44     -179.902620        0.018163
BFGS:   15 12:02:44     -179.902803        0.028171
BFGS:   16 12:02:45     -179.903137        0.040462
BFGS:   17 12:02:45     -179.903585        0.053050
BFGS:   18 12:02:45     -179.903927        0.036758
BFGS:   19 12:02:45     -179.904062        0.020070
BFGS:   20 1

### Pegando novos parâmetros da rede

In [7]:
atoms.cell.cellpar()

array([  7.55246858,   7.55246858,   7.12916593,  90.        ,
        90.        , 120.        ])

Salvando nova estrutura otimizada

In [ ]:
atoms.set_constraint() #Remove constraints 
atoms.wrap()
write('ice_Ih_relaxed.pdb',atoms)

Criando uma supercélula 

In [ ]:
atoms = atoms.repeat((2,2,2))
write('ice_Ih_2x2x2_relaxed.pdb',atoms)

In [9]:
atoms.cell.cellpar()

array([ 15.10493717,  15.10493717,  14.25833186,  90.        ,
        90.        , 120.        ])